In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [27]:
gdf = pd.read_parquet("../data/eval_data_gemma2_stage1_logits.parquet")
gtta_df = pd.read_parquet("../data/eval_data_gemma2_stage1_logits_tta.parquet")

cols = ["id", "prompt", "response_a", "response_b", "response_b", "model_a", "model_b", "language", "split", "source", "winner"]
gmerged_df = pd.merge(gdf, gtta_df, on=cols, suffixes=("", "_tta"))
gmerged_df['logits_tta'] = gmerged_df['logits_tta'].apply(lambda x: x[::-1])
gmerged_df

,id,prompt,response_a,response_b,winner,model_a,model_b,language,split,source,logits,logits_tta
0,050475156d71ce0b29aadb791b7c2637dcc353c90bf3c2...,Consider that a very advanced and self-aware g...,Here's a detailed outline of a Python module a...,Creating a comprehensive simulation of the Orb...,model_b,claude-3-opus-20240229,gpt-4o-mini-2024-07-18,English,valid,current_comp,"[0.6015625, 0.58984375]","[0.4609375, 0.74609375]"
1,0741b308cee0ba4c028ffec8086fcdff01c85db230de99...,hi ! rewrite the following game and make a str...,Certainly! Below is a reworked version of the ...,Sure! Let's transform your text-based adventur...,model_b,yi-lightning-lite,o1-mini,English,valid,current_comp,"[0.423828125, 0.578125]","[0.390625, 0.7734375]"
2,115888577ff36990aa7438275a4657f880673fe722eadc...,Satisfying Psychological Needs Using Luxury Br...,**Literature Review**\n\n**2.1. Definition of ...,Here is the literature review with a similar s...,model_b,reka-core-20240904,llama-3.1-405b-instruct-bf16,English,valid,current_comp,"[1.046875, 0.23828125]","[0.6953125, 0.388671875]"
3,11984059c5138393efb476c2d3feacebb45be1014e6f4c...,Title: Analyzing the Impact of Free Market Exc...,**Title**: Analyzing the Impact of Free Market...,## Analyzing the Impact of Free Market Exchang...,model_a,o1-preview,gemma-2-2b-it,English,valid,current_comp,"[2.09375, -0.71875]","[1.6328125, -0.65625]"
4,14017a37962fdc186946866a37df2abfdc4cc2a016369e...,use c++20 and make it class with oop: inline u...,Below is a refactored version of the provided ...,To convert the provided C-style code into a C+...,model_b,llama-3.2-3b-instruct,chatgpt-4o-latest-20240903,English,valid,current_comp,"[0.2138671875, 0.80859375]","[0.11376953125, 1.1015625]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4834,8fd449e93f6cfe83e15007371518a9fae37f3ffd466de9...,Ии приветик,Привет! Как дела?,Приветик! Как я могу помочь тебе сегодня? 😊🤖,model_a,gemini-1.5-flash-8b-001,glm-4-plus,Russian,valid,current_comp,"[-0.1689453125, 0.93359375]","[0.49609375, 0.388671875]"
4835,58fac0d8d980831df3a09a46aa070a7c99e3ef6864b9ff...,How can I found out the solution to the world?...,Unanswerable.,Research,model_b,gemini-1.5-pro-002,grok-2-2024-08-13,English,valid,current_comp,"[0.69921875, 0.4140625]","[0.60546875, 0.251953125]"
4836,d62c05368101623351bb2b9bf09934f94852dceb60acdd...,How can I found out the solution to the world?...,Reflection.,Collaboration.,model_a,yi-lightning,command-r-08-2024,English,valid,current_comp,"[0.154296875, 0.734375]","[0.59375, 0.318359375]"
4837,17f18300faa205116513ae2310cc4f5f60fe3d970148e6...,eae,Eae! Como posso ajudar você hoje?,Eae! Como tá?,model_b,qwen-plus-0828,chatgpt-4o-latest-20240903,Basque,valid,current_comp,"[0.32421875, 0.55078125]","[0.70703125, 0.076171875]"


In [83]:
w = 0.6
gemma_logits = w * np.array(gmerged_df['logits'].tolist()) + (1 - w) * np.array(gmerged_df['logits_tta'].tolist())
gemma_preds = np.argmax(gemma_logits, axis=-1)
ground_truth = gmerged_df['winner'].map({"model_a": 0, "model_b": 1}).values
gemma_acc = accuracy_score(ground_truth, gemma_preds)
gemma_acc

0.707997520148791

In [55]:
ldf = pd.read_parquet("../data/eval_data_llama3_stage1_logits.parquet")
ltta_df = pd.read_parquet("../data/eval_data_llama3_stage1_logits_tta.parquet")

cols = ["id", "prompt", "response_a", "response_b", "response_b", "model_a", "model_b", "language", "split", "source", "winner"]
lmerged_df = pd.merge(ldf, ltta_df, on=cols, suffixes=("", "_tta"))
lmerged_df['logits_tta'] = lmerged_df['logits_tta'].apply(lambda x: x[::-1])
lmerged_df.head(2)

,id,prompt,response_a,response_b,winner,model_a,model_b,language,split,source,logits,logits_tta
0,115888577ff36990aa7438275a4657f880673fe722eadc...,Satisfying Psychological Needs Using Luxury Br...,**Literature Review**\n\n**2.1. Definition of ...,Here is the literature review with a similar s...,model_b,reka-core-20240904,llama-3.1-405b-instruct-bf16,English,valid,current_comp,"[0.76953125, 0.369140625]","[1.0703125, 0.11474609375]"
1,11984059c5138393efb476c2d3feacebb45be1014e6f4c...,Title: Analyzing the Impact of Free Market Exc...,**Title**: Analyzing the Impact of Free Market...,## Analyzing the Impact of Free Market Exchang...,model_a,o1-preview,gemma-2-2b-it,English,valid,current_comp,"[2.46875, -0.71875]","[2.890625, -1.0703125]"


In [68]:
llama_logits = 0.5 * np.array(lmerged_df['logits'].tolist()) + 0.5 * np.array(lmerged_df['logits_tta'].tolist())
llama_preds = np.argmax(llama_logits, axis=-1)
ground_truth = lmerged_df['winner'].map({"model_a": 0, "model_b": 1}).values

In [69]:
llama_acc = accuracy_score(ground_truth, llama_preds)
llama_acc

0.6997313494523661

In [50]:
# merge gemma and llama
merged_df = pd.merge(gmerged_df, lmerged_df, on=cols, suffixes=("_gemma", "_llama"))
merged_df["logits_e_gemma"] = ((np.array(merged_df["logits_gemma"].tolist()) + np.array(merged_df["logits_tta_gemma"].tolist())) / 2).tolist()
merged_df["logits_e_llama"] = ((np.array(merged_df["logits_llama"].tolist()) + np.array(merged_df["logits_tta_llama"].tolist())) / 2).tolist()

In [61]:
# find best weights
best_acc = 0
best_weights = None
for w in np.linspace(0, 1, 100):
    ensemble_logits = w * np.array(merged_df["logits_e_gemma"].tolist()) + (1 - w) * np.array(merged_df["logits_e_llama"].tolist())
    ensemble_preds = np.argmax(ensemble_logits, axis=-1)
    acc = accuracy_score(ground_truth, ensemble_preds)
    if acc > best_acc:
        best_acc = acc
        best_weights = w

In [62]:
best_acc, best_weights

(0.7108906798925397, 0.787878787878788)

In [74]:
w = 0.7878
ensemble_logits = w * np.array(merged_df["logits_e_gemma"].tolist()) + (1 - w) * np.array(merged_df["logits_e_llama"].tolist())
ensemble_preds = np.argmax(ensemble_logits, axis=-1)
accuracy_score(ground_truth, ensemble_preds)

0.7108906798925397

In [11]:
# ensure tta works correctly
df = pd.read_parquet("../data/lmsys1m_125k_pseudo_label_gemma2stage1.parquet").sort_values("id")
df

,id,prompt,response_a,response_b,model_a,model_b,language,winner,logits
2046,0000c48f341d489591e3bbeca1821926,¿Cómo hago una petición Ajax en django con jQu...,Para hacer una petición Ajax en Django con jQu...,¡Hola! ¡Claro que sí! Aquí te puedo ayudar con...,Qwen/Qwen2-1.5B-Instruct,llama-2-13b-chat,Spanish,model_b,"[-0.06689453125, 0.87109375]"
109937,0000ce95d8dd4aa6bca0e792b767d550,你好,你好！有什么我能为你效劳的吗？,Hello! How can I help you today? If you have a...,vicuna-13b,mistralai/Mistral-7B-Instruct-v0.3,Chinese,model_a,"[1.0390625, 0.0771484375]"
120267,000174aa3722401cae71c6b3197a82e0,你好,你好！我是中文聊天机器人。如何可以幫助你？,你好！有什么我能帮助你的吗？,meta-llama/Llama-3.1-8B-Instruct,vicuna-13b,Chinese,model_b,"[0.09130859375, 0.73828125]"
85320,0001c758568b4baba4b84389679fbe85,Привет,"Здравствуйте! Пожалуйста, скажите, как могу по...",Привет! Как я могу Вам помочь?,mistralai/Mistral-7B-Instruct-v0.2,vicuna-13b,Russian,model_b,"[0.04248046875, 0.828125]"
119159,00026c3ac57b47e7bf4e4c0ac81c8082,hi,Hello! How can I help you today?,Hello! 👋 How can I help you today? 😊,vicuna-13b,google/gemma-2-9b-it,unknown,model_b,"[0.1015625, 0.67578125]"
...,...,...,...,...,...,...,...,...,...
103045,fffe559651f241bdb473eb396ccdf042,zxcvzxcv,It seems like you're typing some random charac...,"I'm sorry, I don't understand what you are try...",meta-llama/Llama-3.2-3B-Instruct,gpt-3.5-turbo,Danish,model_a,"[0.671875, 0.365234375]"
106644,fffe80dc4e37411fa4355236f52bcd07,Hiya,Hello! How can I assist you today?,Hello! How can I help you today? If you have a...,chatglm-6b,Nexusflow/Starling-LM-7B-beta,Waray,model_b,"[0.373046875, 0.48046875]"
114954,fffed7463ef64edaac3d247ad0d0918b,你好,你好！有什么我能为你效劳的吗？,Ni hao! Kamusta! Hello! How can I help you to...,vicuna-13b,mistralai/Mixtral-8x7B-Instruct-v0.1,Chinese,model_a,"[1.171875, -0.07373046875]"
29808,ffff3da3bc7f4493b93b370bc0bab73e,I have submitted a paper on the topic of trust...,"Dear Reviewers,\n\nThank you for your valuable...",In response to the question about the meaning ...,vicuna-33b,Nexusflow/Starling-LM-7B-beta,unknown,model_a,"[0.71875, 0.234375]"


In [14]:
df_tta = pd.read_parquet("../data/lmsys1m_125k_pseudo_label_gemma2stage1_tta.parquet").sort_values("id")
df_tta[df_tta["id"] == "000c42ef0246417283d9826d4ffd9f57"]


,id,prompt,response_a,response_b,model_a,model_b,language,winner,logits
1239,000c42ef0246417283d9826d4ffd9f57,素人がスナックを始めて設けるには,素人はスナックを始める前に設ける必要がある。 その設定は彼女の姿勢、姿勢の固有なライティング...,A great topic! 🍿 Starting a snack business as ...,alpaca-13b,meta-llama/Meta-Llama-3-8B-Instruct,Japanese,model_b,"[-0.98828125, 2.125]"


In [15]:
df[df["id"] == "000c42ef0246417283d9826d4ffd9f57"]

,id,prompt,response_a,response_b,model_a,model_b,language,winner,logits
31557,000c42ef0246417283d9826d4ffd9f57,素人がスナックを始めて設けるには,素人はスナックを始める前に設ける必要がある。 その設定は彼女の姿勢、姿勢の固有なライティング...,A great topic! 🍿 Starting a snack business as ...,alpaca-13b,meta-llama/Meta-Llama-3-8B-Instruct,Japanese,model_b,"[-1.1953125, 2.140625]"
